# Recommendation systems
Welcome to your first homework assignment about recommendation systems.


## The Cold Start Problem 

The colaborative filtering method discussed in class does not address the problem of new user or new movies. What prediction would you use in these cases:

* A new user but a known movie
* A new movie and a known user
* A new user and new movie

* A new user but a known movie: in the user-based collaborative filtering, find the most correlated user to the new user and refer to that user's rating to the movie
* A new movie and a known user: in the movie-based collaborative filtering, find the most correlated known movie to the new movie and refer to the user's rating to the movie
* A new user and new movie: find the most correlated movie to the new movie and most correlated user to the new user, refer to the old user's rating to the old movie

## Matrix Factorization with bias
We want to extend the Matrix Factorization model discussed in class to add a "bias" parameter for each user and another "bias" parameter for each movie.  For the problem in class we had the parameters matrix $U$ and $V$, we are adding $u^0$ which is a vector of dimension $n_u$ and $v^0$ which is a vector of dimension $n_m$. The equations

$$\hat{y}_{ij} = u_{0i} + v_{0j} + u_i \cdot v_j  $$ 
 
(a) How many weights (parameters) are we fitting for this problem?

(b) Write the gradient descent equations for this problem.

(a) number of parameters: $$ n_{m}*K + n_{u}*K +n_{m}+n_{u} $$
(b) gradient descent equations 
$$u_{ik} = u_{ik}+ \frac{2\eta}{N}\sum_{j:r_{ij = 1}}^{}(y_{ij} - u_{i}\times v_{j} - u_{0i} - v_{0j})\times v_{jk}$$ $$and$$ 
$$v_{ik} = v_{ik}+ \frac{2\eta}{N}\sum_{j:r_{ij = 1}}^{}(y_{ij} - u_{i}\times v_{j} - u_{0i} - v_{0j} )\times u_{jk}$$ $$and$$
$$u_{0i} = u_{0i}+ \frac{2\eta}{N}\sum_{j:r_{ij = 1}}^{}(y_{ij} - u_{i}\times v_{j} - u_{0i} - v_{0j})$$ 
$$and$$ 

$$v_{0j} = v_{0j}+ \frac{2\eta}{N}\sum_{j:r_{ij = 1}}^{}(y_{ij} - u_{i}\times v_{j} - u_{0i} - v_{0j})$$ 

# Collaborative Filtering with Stochastic Gradient Descent 

In this part of the assignment you will build a collaborative filtering model to predict netflix ratings.  This assignment will step you through how to do this using stochastic gradient descent. The data for this assignment is ...

**Instructions:**
- Do not use loops (for/while) in your code, unless the instructions explicitly ask you to do so.
- DO NOT change paths (-3 points)
- DO NOT submit data to github (-2 points)

**You will learn to:**
- Build the general architecture of a learning algorithm, including:
    - Encoding rating data
    - Initializing parameters
    - Calculating the cost function
    - Calculating gradient
    - Using an optimization algorithm (gradient descent) 
    - Predicting on new data
- Putting it all together.

In [1]:
import numpy as np
import pandas as pd

## Encoding rating data
Here are our very small subset of fake data to get us started.

In [2]:
# The first row says that user 1 reated movie 11 with a score of 4
!cat tiny_training2.csv 

userId,movieId,rating
11,1,4
11,23,5
2,23,5
2,4,3
31,1,4
31,23,4
4,1,5
4,3,2
52,1,1
52,3,4
61,3,5
7,23,1
7,3,3


In [3]:
# here is a handy function from fast.ai
def proc_col(col):
    """Encodes a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [4]:
def encode_data(df):
    """Encodes rating data with continous user and movie ids using 
    the helpful fast.ai function from above.
    
    Arguments:
      train_csv: a csv file with columns user_id,movie_id,rating 
    
    Returns:
      df: a dataframe with the encode data
      num_users
      num_movies
      
    """
    df["userId"] = proc_col(df["userId"])[1]
    df["movieId"] = proc_col(df["movieId"])[1]
    num_users = proc_col(df["userId"])[2]
    num_movies = proc_col(df["movieId"])[2]
    #raise NotImplementedError()
    return df, num_users, num_movies

In [5]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)

In [6]:
assert(num_users == 7)

In [7]:
assert(num_movies == 4)

## Initializing parameters

In [9]:
def create_embedings(n, K):
    """ Create a numpy random matrix of shape n, K
    
    The random matrix should be initialized with uniform values in (0, 6/K)
    Arguments:
    
    Inputs:
    n: number of items/users
    K: number of factors in the embeding 
    
    Returns:
    emb: numpy array of shape (n, num_factors)
    """
    np.random.seed(3)
    emb = 6*np.random.random((n, K)) / K
    return emb

# here is an example on how the prediction matrix would look like with 7 users and 5 movies
np.dot(create_embedings(7,3), create_embedings(5,3).transpose())

array([[ 3.55790894,  4.69774849,  0.92361109,  1.58739544,  3.00593239],
       [ 4.69774849,  7.44656163,  1.18135616,  2.64524868,  4.74559066],
       [ 0.92361109,  1.18135616,  0.24548062,  0.34025121,  0.69616965],
       [ 1.58739544,  2.64524868,  0.34025121,  1.61561   ,  2.41361975],
       [ 3.00593239,  4.74559066,  0.69616965,  2.41361975,  3.82505541],
       [ 2.02000808,  3.29656257,  0.43174569,  2.065911  ,  3.07264619],
       [ 2.07691001,  3.02887291,  0.53270924,  1.02482544,  1.90251125]])

## Encoding Y as a sparse matrix
This code helps you encode a $Y$ as a sparse matrix from the dataframe. 

In [10]:
from scipy import sparse
def df2matrix(df, nrows, ncols, column_name="rating"):
    """ Returns a sparse matrix constructed from a dataframe
    
    This code assumes the df has columns: MovieID,UserID,Rating
    """
    values = df[column_name].values
    ind_movie = df['movieId'].values
    ind_user = df['userId'].values
    return sparse.csc_matrix((values,(ind_user, ind_movie)),shape=(nrows, ncols))

In [11]:
df = pd.read_csv("tiny_training2.csv")
df, num_users, num_movies = encode_data(df)
Y = df2matrix(df, num_users, num_movies)

In [12]:
print(Y)

  (0, 0)	4
  (2, 0)	4
  (3, 0)	5
  (4, 0)	1
  (0, 1)	5
  (1, 1)	5
  (2, 1)	4
  (6, 1)	1
  (1, 2)	3
  (3, 3)	2
  (4, 3)	4
  (5, 3)	5
  (6, 3)	3


In [13]:
def sparse_multiply(df, emb_user, emb_movie):
    """ This function returns U*V^T element wise multi by R as a sparse matrix.
    
    It avoids creating the dense matrix U*V^T
    """
    df["Prediction"] = np.sum(emb_user[df["userId"].values]*emb_movie[df["movieId"].values], axis=1)
    return df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="Prediction")

## Calculating the cost function

In [14]:
# Use vectorized computation for this function. No loops!
# Hint: use df2matrix and sparse_multiply
def cost(df, emb_user, emb_movie):
    """ Computes mean square error
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      error(float): this is the MSE
    """
    prediction = sparse_multiply(df,emb_user,emb_movie).data
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0]).data
    diff = (Y - prediction)**2
    error = np.mean(diff)  
    #raise NotImplementedError()
    return error

In [15]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
error = cost(df, emb_user, emb_movie)
assert(np.around(error, decimals=2) == 2.23)

## Calculating gradient

In [17]:
def finite_difference(df, emb_user, emb_movie, ind_u=None, ind_m=None, k=None):
    """ Computes finite difference on MSE(U, V).
    
    This function is used for testing the gradient function. 
    """
    e = 0.000000001
    c1 = cost(df, emb_user, emb_movie)
    K = emb_user.shape[1]
    x = np.zeros_like(emb_user)
    y = np.zeros_like(emb_movie)
    if ind_u is not None:
        x[ind_u][k] = e
    else:
        y[ind_m][k] = e
    c2 = cost(df, emb_user + x, emb_movie + y)
    return (c2 - c1)/e

In [18]:
def gradient(df, Y, emb_user, emb_movie):
    """ Computes the gradient.
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      d_emb_user
      d_emb_movie
    """
    prediction = sparse_multiply(df,emb_user,emb_movie)
    delta = Y.toarray() - prediction.toarray()
    d_emb_user =  (-2.0)*np.dot(delta/len(prediction.data), emb_movie)
    d_emb_movie = (-2.0)*np.dot((delta/len(prediction.data)).T,emb_user)
    return d_emb_user, d_emb_movie
    #raise NotImplementedError()

In [19]:
K = 3
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
grad_user, grad_movie = gradient(df, Y, emb_user, emb_movie)

In [20]:
grad_user

array([[-0.12243115, -0.17937243, -0.12292687],
       [ 0.31426962,  0.55623039,  0.6459187 ],
       [-0.96440297, -1.44475001, -1.05269757],
       [-0.63049179, -0.74711157, -0.35949076],
       [ 0.12479118,  0.42249281, -0.04343876],
       [-0.39796164, -0.02697214, -0.41242749],
       [ 0.0509932 ,  0.53928159,  0.28188905]])

In [21]:
user=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_u=user, k=i) for i in range(K)])
assert(np.all(np.abs(grad_user[user] - approx) < 0.0001))

In [22]:
movie=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_m=movie, k=i) for i in range(K)])
assert(np.all(np.abs(grad_movie[movie] - approx) < 0.0001))

In [23]:
movie=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_m=movie, k=i) for i in range(K)])
assert(np.all(np.abs(grad_movie[movie] - approx) < 0.0001))

## Using gradient descent with momentum

In [34]:
# you can use a for loop to iterate through gradient descent
def gradient_descent(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None):
    """ Computes gradient descent with momentum (0.9) for a number of iterations.
    
    Prints training cost and validation cost (if df_val is not None) every 50 iterations.
    
    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    emb_user_momen, emb_movie_momen = gradient(df, Y, emb_user, emb_movie)
    
    for i in range(iterations):
        emb_user_grad, emb_movie_grad = gradient(df, Y, emb_user, emb_movie)
        
        emb_user_momen = 0.9*emb_user_momen + 0.1*emb_user_grad
        emb_user = emb_user- learning_rate*emb_user_momen
        
        emb_movie_momen = 0.9*emb_movie_momen + 0.1*emb_movie_grad
        emb_movie = emb_movie- learning_rate*emb_movie_momen
        
        
        if (i+1)%50 == 0:
            print "training cost:",cost(df, emb_user, emb_movie)
            if df_val is None:
                print "None"
            else: 
                print "validation cost:", cost(df_val,emb_user, emb_movie)
       
    
    #raise NotImplementedError()
    return emb_user, emb_movie

In [35]:
emb_user = create_embedings(num_users, 3)
emb_movie = create_embedings(num_movies, 3)
emb_user, emb_movie = gradient_descent(df, emb_user, emb_movie, iterations=200, learning_rate=0.01)

training cost: 1.70136430619
None
training cost: 0.974870517206
None
training cost: 0.699145565693
None
training cost: 0.526562109415
None


In [36]:
train_mse = cost(df, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.53)

## Predicting on new data
Now we should write a function that given new data is able to predict ratings. First we write a function that encodes new data. If a new user or item is present that row should be remove. Collaborative Filtering is not good at handling new users or new items. To help with this task, you could write a an auxiliary function similar to `proc_col`.

In [126]:
def proc_validation(val, col, train):
    uniq = train[col].unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return np.array([name2idx[x] for x in val[col]]), len(uniq)

def encode_new_data(df_val, df_train):
    """ Encodes df_val with the same encoding as df_train.
    Returns:
    df_val: dataframe with the same encoding as df_train
    """
    df_val = df_val[df_val['movieId'].isin(df_train['movieId'])]
    df_val = df_val[df_val['userId'].isin(df_train['userId'])]
    df_val['userId'], num_users = proc_validation(df_val, 'userId', df_train)
    df_val['movieId'], num_movies = proc_validation(df_val, 'movieId', df_train)
    return df_val

In [127]:
df_t = pd.read_csv("tiny_training2.csv")
df_v = pd.read_csv("tiny_val2.csv")
df_v = encode_new_data2(df_v, df_t)

In [128]:
assert(len(df_v.userId.unique())==2)

In [129]:
assert(len(df_v) == 2)

## Putting it all together
For this part you should get data from here
`wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip`

In [120]:
# Don't change this path use a simlink if you have the data somewhere else
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
# sorting by timestamp take as validation data the most recent data doesn't work so let's just take 20%
# at random
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()
df_train, num_users, num_movies = encode_data(train)
df_val = encode_new_data2(val, train)
print(len(val), len(df_val))

(20205, 16794)


In [121]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=2000, learning_rate=1, df_val=df_val)

training cost: 9.98836344217
validation cost: 11.7733625978
training cost: 7.22565025945
validation cost: 10.7234729191
training cost: 5.22643461835
validation cost: 9.61633277535
training cost: 4.07667755518
validation cost: 8.65862114337
training cost: 3.34174315552
validation cost: 7.86071228897
training cost: 2.83263058825
validation cost: 7.1894528935
training cost: 2.46393754738
validation cost: 6.62056457868
training cost: 2.18758530287
validation cost: 6.13541108102
training cost: 1.97441970687
validation cost: 5.71877432506
training cost: 1.80592801063
validation cost: 5.35834004196
training cost: 1.66992604337
validation cost: 5.0442523121
training cost: 1.55815302884
validation cost: 4.7686532689
training cost: 1.46485885392
validation cost: 4.52526998551
training cost: 1.3859424916
validation cost: 4.30907091825
training cost: 1.3184119916
validation cost: 4.11599240998
training cost: 1.26003836732
validation cost: 3.94272635676
training cost: 1.20912909979
validation cost:

In [122]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

(0.7665730026459856, 2.2704020649286067)


In [123]:
train_mse = cost(df_train, emb_user, emb_movie)
assert(np.around(train_mse, decimals=2) == 0.77)

In [124]:
val_mse = cost(df_val, emb_user, emb_movie)
assert(np.around(val_mse, decimals=2) == 2.27)

In [125]:
np.around(val_mse, decimals=2)

2.27

#  Advanced Recommendation (Optional)
Use Regression / Random forest to add other features to your recommendation algorithm. Here are the steps:

* Here are potential features: 
    * user embeding, movie embeding (from the hw)
    * movie genres
    * daypart features
    
Other extentions: add regularization to gradient descent

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()